# Exploration dataset

In [ ]:
import os
import sys
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(stream=sys.stdout, format='',
                level=logging.INFO, datefmt=None)
logger = logging.getLogger('RCD-exploration')

In [ ]:
#https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False 

import os
import sys
from pathlib import Path
import json

import numpy as np

#import tensorflow as tf
#import tensorflow_hub as hub

import PIL
from PIL import Image, ImageDraw
from io import BytesIO
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (8.0, 6.0)
plt.rcParams['axes.titlepad'] = 20 

from IPython.display import display, Markdown, HTML

#print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__)
#print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Download dataset

In [ ]:
def preprocess_image(image):
  image = np.array(image)
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)
  return image

def load_image_from_path(img_path: str) -> tf.Tensor:
  """Returns an image with shape [1, height, width, num_channels]."""
  image = Image.open(Path(img_path))
  image = preprocess_image(image)
  return image

def show_image(image, title=''):
  image_size = image.shape[1]
  w = (image_size * 6) // 320
  plt.figure(figsize=(w, w))
  plt.imshow(image[0], aspect='equal')
  plt.axis('off')
  plt.title(title)
  plt.show()

In [ ]:
CLASSES = ['good', 'poor', 'verypoor', 'satis']
img_ls = []
for c in CLASSES:
    PATH_GOOD_IMG = f"../dataset/sih_road_dataset/{c}/{c}_001.jpg"
    img_ls.append(load_image_from_path(PATH_GOOD_IMG))
  
fig, axes = plt.subplots(nrows=2, ncols=2)
for i, img in enumerate(img_ls):
    offset = i % 2
    # Plotting each image
    axes[i//2][offset].imshow(img[0])
    # Set labels and turning off axes 
    axes[i//2][offset].set_title(CLASSES[i].upper())
    axes[i//2][offset].axis('off')

# Dataset stats

In [ ]:
import pandas as pd

stats_df = pd.DataFrame(columns=['class', '#'])
img_sizes = []
img_path = []
for n, c in enumerate(CLASSES):
    # get classes info
    dir_path = Path(f'../dataset/sih_road_dataset/{c}')
    n_imgs = len(list(dir_path.glob('*.jpg')))
    stats_df.loc[n, 'class'] = c
    stats_df.loc[n, '#'] = n_imgs
    # get images size
    for p in dir_path.iterdir():
        if str(p).endswith('.jpg'):
            # [(img1_width, img1_height),..., (imgN_width, imgN_height)]
            img_sizes.append(Image.open(p).size)
            img_path.append(p)

img_size_df = pd.DataFrame.from_records(img_sizes)
img_size_df.columns = ['width', 'height']
img_size_df.index = img_path
stats_df
img_size_df.describe()

In [ ]:
img_size_df.head()

In [ ]:
img1 = Image.open(img_size_df.index[0]).resize((512, 512))
img2 = Image.open(img_size_df.index[10]).resize((512, 512))
img1.show()
img2.show()
Image.blend(img1, img2, alpha=0.5).resize((512,512)).show()

In [ ]:
mask = Image.new('L', size=img1.size, color=0)
mask.show()
draw = ImageDraw.Draw(mask)
draw.ellipse((250, 350, 300, 400), fill=255)
mask.show()
img_composed = Image.composite(img1, img2, mask)
img_composed.show()